In [8]:
import numpy as np
from activation_helper import sigmoid
from make_dataset_helper import prepare_dataset
    

def initialize_parameter(layer_dims: list) -> dict:
    # generate parameter W, b
    parameter = {}
    for i in range(1, len(layer_dims)):
        parameter["W" + str(i)] = np.random.randn(layer_dims[i], layer_dims[i - 1]) * 0.01
        parameter["b" + str(i)] = np.zeros((layer_dims[i], 1))
    
    assert(parameter["W1"].shape == (layer_dims[1], layer_dims[0]))
    
    # test done
    return parameter
        
    

def main(X, y, layer_dims, learning_rate=0.01):
    parameter = initialize_parameter(layer_dims)
    print(parameter)
    


if __name__ == '__main__':
    X_train, X_test, y_train, y_test = prepare_dataset()
    layer_dims=[X_train.shape[0],3,2,1]
    main(X_train, y_train, layer_dims, learning_rate=0.01)
    

{'W1': array([[-0.00969058,  0.00914837],
       [ 0.00345236,  0.01084597],
       [ 0.00260346,  0.00327564]]), 'b1': array([[ 0.],
       [ 0.],
       [ 0.]]), 'W2': array([[ 0.01050658, -0.00500637,  0.00231161],
       [ 0.00711537, -0.00136863,  0.01885059]]), 'b2': array([[ 0.],
       [ 0.]]), 'W3': array([[ 0.00396756,  0.01587136]]), 'b3': array([[ 0.]])}
